# Physics-Informed-Neural-Network to solve Partial Differential Equation
- Aim: To compare performance with Finite-Volume Implicit Numerical Scheme

## Problems with traditional methods: -
- High computational costs
- Boundary problems
- Computational costs for mesh generation
- Scaling for complex systems
- Numerical Instability

## Advantages of Neural Networks: -
- Adaptive learning- hence, may approximate function more efficiently
- Auto Differentiation- Key aspect of NN (gradient descent) that can also be applied to PDE solving for precise derivative calculations more efficienty
- PINNs use a cumulative loss function that ensures the computed answer complies with both- the data and the laws of physics that govern the PDE 
- Hence, appear to be more accurate approximations by embedding physics
- Consequently, should be more data efficient

## Risks to explore: -
- Training complexity
- Overfitting
- Numerical Stability

In [181]:
# Imports
import tensorflow as tf
import numpy as np

from tensorflow.keras import layers, models, initializers
from keras_tuner import HyperModel, RandomSearch

import math as m 

# Data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)

## Mathematical Setup: -
### Idea: -

We construct a Neural Network approximation,
$$
u_\theta(t, x) \approx u(t,x),
$$

where $u_\theta : [0, T] \times \mathcal{D} \to \mathbb{R}$ denotes a function approximated by a neural network with parameters $\theta$.

## Example: Burger's Equation

$$
\begin{aligned}
    \partial_t u + u.\partial_x u - v.\partial_{xx} u &= 0, \quad && \quad (t, x) \in (0, 1] \times (-1, 1);\\
   u(0, x) &= -\sin(\pi.x), \quad && \quad x \in [-1,1];\\
   u(t, -1) &= u(t, 1) = 0, \quad && \quad t \in (0,1];
\end{aligned}
$$

In [182]:
# Constants
alpha = 1  # v
pi = tf.constant(m.pi)

# Initial condition
def u_0(x):
    return -tf.sin(pi*x)    # u_0(x) = sin(pi.x)

# PDE residual using TensorFlow AutoDiff
def pde_residual(model, t, x):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([t, x])
        u = model(x)
        u_t = tape.gradient(u, t)
    u_x = tape.gradient(u, x)
    u_xx = tape.gradient(u_x, x)
    del tape

    return u_t - alpha*u_xx

In [183]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(9,6))
# plt.scatter(t_0, x_0, c=u_0, marker='X', vmin=-1, vmax=1)
# plt.scatter(t_b, x_b, c=u_b, marker='X', vmin=-1, vmax=1)
# plt.scatter(t_r, x_r, c='r', marker='.', alpha=0.1)

# plt.xlabel('$t$'); plt.ylabel('$x$')
# plt.title('Mesh')

## Network Architecture

In [184]:
# Activation Function
activation = 'tanh'

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#PINN model
class PINNModel(HyperModel):
    def build(self, hp):
        """
        Build a PINN model for Burgers' equation with input shape (t, x) and output shape u(t, x).

        Returns:
            Network model
        """

        model = models.Sequential()

        # 1 neuron input layer: (x)
        model.add(layers.InputLayer((1,))) 
        # Hidden layers
        for i in range(hp.Int('num_layers', 1, 32)):
            model.add(layers.Dense(
                units= hp.Int('units_' + str(i), 32, 512, step=32),
                activation = activation,
                kernel_initializer = 'glorot_normal'
            ))
        # 1 neuron output layer: u(x)
        model.add(layers.Dense(1))  

        model.compile(
            optimizer=optimizer,
            loss=lambda y_true, y_pred: pinn_loss(model, t, x, x_bc, u_bc)
        ) 

        return model

# Loss function that enforces physical lalws (PDE and boundary conditions)
def pinn_loss(model, t, x, x_bc, u_bc):
    pde_loss = tf.reduce_mean(tf.square(pde_residual(model, t, x)))
    bc_loss = tf.reduce_mean(tf.square(model(x_bc) - u_bc))
    
    return pde_loss + bc_loss

### Training

In [185]:
# Custom training step
@tf.function
def train_step(model, t, x, x_bc, u_bc):
    with tf.GradientTape() as tape:
        loss = pinn_loss(model, t, x, x_bc, u_bc)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training data
def generate_data(num_samples):
    x = tf.random.uniform((num_samples, 1), 0, 1)
    u_true = u_0(x)
    return u_true, x

# Generate boundary condition data
def generate_boundary_data(num_samples):
    x_bc = tf.concat([tf.zeros((num_samples // 2, 1)), tf.ones((num_samples // 2, 1))], axis=0)
    u_bc = tf.concat([tf.zeros((num_samples // 2, 1)), tf.zeros((num_samples // 2, 1))], axis=0)
    return x_bc, u_bc

# Data
num_samples = 201
t, x = tf.convert_to_tensor(np.arange(0, 2.01, 0.01)[:201]), tf.convert_to_tensor(np.arange(0, 1.01, 0.005)[:201])
u_train, x_train = generate_data(num_samples)
x_bc, u_bc = generate_boundary_data(num_samples)


### Hyperparameter Tuning

In [186]:
# Hyperparameter tuning
hypermodel = PINNModel()
tuner = RandomSearch(
    hypermodel,
    objective='loss',
    max_trials=100,
    executions_per_trial=1,
    directory='pinn_tuning',
    project_name='pinn_burgers'
)

# Perform the search
tuner.search(x_train, u_train, epochs=100, validation_data=None)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

Reloading Tuner from pinn_tuning\pinn_burgers\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
6                 |1                 |num_layers
160               |224               |units_0
256               |None              |units_1
256               |None              |units_2
128               |None              |units_3
32                |None              |units_4
320               |None              |units_5
128               |None              |units_6
128               |None              |units_7
352               |None              |units_8
288               |None              |units_9
224               |None              |units_10
288               |None              |units_11
416               |None              |units_12
160               |None              |units_13
320               |None              |units_14
256               |None              |units_15
352               |None              |units_16
352               |None 

Traceback (most recent call last):
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\89\AppData\Roaming\Python\Python312\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.

### Best Model

In [ ]:
# Train the best model
for epoch in range(1000):
    loss = train_step(best_model, t_train, x_train, t_bc, x_bc, u_bc)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

### Model Usage

### Simulation Setup

In [ ]:

# # Simulation Parameters

# # Number of points
# N_0 = 64
# N_b = 64
# N_r = 8320


# # Mesh
# tmin, tmax = 0., 1.
# xmin, xmax = -1., 1.


# # Lower bounds
# lb = tf.constant([tmin, xmin], dtype=DTYPE)
# # Upper bounds
# ub = tf.constant([tmax, xmax], dtype=DTYPE)


# # Draw uniform sample points for initial boundary data
# tf.random.set_seed(123)     #seed for reproducible results
# t_0 = tf.ones((N_0,1), dtype=DTYPE)*lb[0]
# x_0 = tf.random.uniform((N_0,1), lb[1], ub[1], dtype=DTYPE)
# X_0 = tf.concat([t_0, x_0], axis=1)


# # Boundary data
# t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
# x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b,1), 0.5, dtype=DTYPE)
# X_b = tf.concat([t_b, x_b], axis=1)

# X_data = [X_0, X_b]

# # Draw uniformly sampled collocation points
# t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
# x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
# X_r = tf.concat([t_r, x_r], axis=1)

# # u_0 = fun_u_0(x_0)          # Initial data
# # u_b = fun_u_b(t_b, x_b)     # Boundary data

# # u_data = [u_0, u_b]

## Mesh
- collocation points: red circles
- boundary and initial conditions: cross marks (colour indicates value)

In [ ]:
# # Evaluate the model
# u_pred = best_model(tf.concat([t_train, x_train], axis=1))
# print("Final loss:", pinn_loss(best_model, t_train, x_train, t_bc, x_bc, u_bc).numpy(), u_pred)